In [1]:
#安装需要的各种类库
!pip install transformers==4.34.1 --upgrade #  支持 flash_attn_func
!pip install tensorrt
!pip install llmtuner==0.1.3   #训练依赖的类库
!pip install bitsandbytes>=0.39.0  # 量化操作需要的类库！
#FlashAttenttion-2 目前只支持 A100 或 H100  其他GPU不支持
!pip install packaging
!pip install ninja
!pip install flash-attn --no-build-isolation
!pip install openai
!pip install cohere
!pip install transformers_stream_generator
!pip install llm-rs --quiet
!pip install langchain --quiet
!pip install llm-rs[langchain] --quiet
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17281 sha256=a5bd18c549895d7463322db5c8f70f5d501be1737814dd63aab730a28e78bf22
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 34.3 MB/s eta 0:00:00
     ━━━━━

相关资料：

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import torch
from torch import nn
import numpy as np
import re
from transformers import LlamaTokenizer, LlamaForCausalLM
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, LlamaForSequenceClassification
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)
from tqdm import tqdm

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchmetrics import Accuracy

In [3]:
# model_path = 'openlm-research/open_llama_3b' # https://huggingface.co/openlm-research/open_llama_3b
model_path ='togethercomputer/Llama-2-7B-32K-Instruct'
# num_labels = 4 # 'Positive', 'Neutral', 'Negative', 'Irrelevant' (https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2)
num_labels = 2 # True or False

model = LlamaForSequenceClassification.from_pretrained(model_path, num_labels=num_labels, torch_dtype=torch.float16, device_map='cuda:0') #device_map='cuda:0'
# Tokenize the input: Use the AutoTokenizer class from the transformers library to tokenize your input text. Set the tokenizer options according to your classification task.

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=False,
    trust_remote_code=True,
    padding_side="left",
    pad_token="<|endoftext|>"
)

model.resize_token_embeddings(len(tokenizer)) # https://github.com/huggingface/transformers/issues/1805

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at togethercomputer/Llama-2-7B-32K-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(32002, 4096)

In [4]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"], # https://github.com/huggingface/peft/blob/632997d1fb776c3cf05d8c2537ac9a98a7ce9435/src/peft/utils/other.py#L202
    lora_dropout=0.1,
    bias="none",   #bias="lora_only",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)

In [5]:
def basic_cleaning(text):
    # Capture swear words that are **** out return text
    text=re.sub(r'https?://www\.\S+\.com','',text)
    text=re.sub(r'[^A-Za-z|\s]','',text)
    text=re.sub(r'\*+','swear',text)
    return text

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F" #emoticons
                               u"\U0001F300-\U0001F5FF" #symbols & pictographs
                               u"\U0001F680-\U0001F6FF" #transport & map symbols
                               u"\U0001F1E0-\U0001F1FF" #flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_multiplechars(text):
    text = re.sub(r'(.)\1{3,}',r'\1', text)
    return text

def clean(df):
    for col in ['text']:#,'selected_text']:
        df[col]=df[col].astype(str).apply(lambda x:basic_cleaning(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_emoji(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_html(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_multiplechars(x))
    return df

fetch the liar dataset



In [ ]:
#fetch the liar dataset from net
from datasets import load_dataset

liar_dataset = load_dataset("liar")


import pandas as pd
idx = 0
# show the data in the dataset
# have 3 dataset : train  ,  test  , validation
for key,v in liar_dataset.items():
  print(key)
  print(v)


# Access different dataset splits
train_dataset = liar_dataset["train"]
validation_dataset = liar_dataset["validation"]
test_dataset = liar_dataset["test"]

sample_example = train_dataset[0]
print(" : ",sample_example)
text = sample_example["statement"]
label = sample_example["label"]

print("Text:", text)
print("Label:", label)


# change the dataset
def datasetToCsv(dataset,name):
  labels =[]
  texts=[]
  for row in dataset:
    # print(row['label'])
    ll = row['label']
    if ll==0 or ll==5 or ll==4:
      labels.append('0')
    else:
      labels.append('1')
    texts.append(row['statement'])
  datas_dict={'text':texts,'label':labels}
  datas = pd.DataFrame(datas_dict)
  print(datas.info)
  datas.to_csv(name)

# change the dataset to the different type dataset
datasetToCsv(train_dataset,"liar_train.csv")

datasetToCsv(validation_dataset,"liar_validation.csv")
datasetToCsv(test_dataset,"liar_test.csv")


In [ ]:
train_data = pd.read_csv("./liar_train.csv")
train_test = pd.read_csv('./liar_test.csv')

print(train_data.info)
print(train_test.info)
#Obtain liar data for processing

if 'label' in train_data.columns:
  train_data['Type'] = train_data['label']
  train_data['Type'].astype('str')
if 'label' in train_test.columns:
  train_test['Type'] = train_test['label']
  train_test['Type'].astype('str')
print(train_data.head())

train_data  = train_data[["text",'Type']]
print(train_data.head())

train_test  = train_test[["text",'Type']]
print(train_test.head())
print("shape:: ",train_test.shape)

# setting
eval_data_twitter =train_data.sample(300) #train_test
data_twitter = train_data

earlyStopAtTrainBatchIdx = 2000 # Set these variables to None to train and validate on all the dataset
earlyStopAtValBatchIdx = 299 # Set these variables to None to train and validate on all the dataset


eval_data_twitter

In [34]:
#使用真假新闻的数据
data_twitter_clean = clean(data_twitter)
eval_data_twitter_clean = clean(eval_data_twitter)
eval_data_twitter_clean
data_twitter_clean

,Unnamed: 0,text,Type
1385,2863,Republicans voice growing doubts on US health...,1
852,3896,Reports are surfacing that in the th hour Tru...,0
144,11740,Federal Judge James Robart a George W Bush ap...,0
637,17969,EU says ball entirely in UK court for move to ...,1
822,15049,US President Donald Trump offers to mediate o...,1
...,...,...,...
1285,8650,Michigan Gov Poison Gets Humiliated By Mark Ru...,0
103,15650,Trump in Japan stresses unity on North Korea t...,1
1259,20631,Central African Republic risks return to majo...,1
711,6339,An email obtained by WTVF in Nashville Tennes...,0


In [35]:
class CustomDataset(Dataset):
    def __init__(self, twitterDF):
        self.twitterDF = twitterDF

    def __len__(self):
        return len(self.twitterDF.index)

    def __getitem__(self, idx):
        return np.array([idx])

In [36]:
train_twitter_dataset = CustomDataset(data_twitter_clean)
train_twitter_dataloader = DataLoader(train_twitter_dataset,batch_size=3,shuffle=True)


eval_twitter_dataset = CustomDataset(eval_data_twitter_clean)
eval_twitter_dataloader = DataLoader(eval_twitter_dataset,batch_size=4,shuffle=False)


In [41]:
# Define your training loop
epochs = 8
# Let's just train on a small batch of the dataset and see the validation score
#fakenews dataset

earlyStopAtTrainBatchIdx = 450 # Set these variables to None to train and validate on all the dataset
earlyStopAtValBatchIdx = 30 # Set these variables to None to train and validate on all the dataset


#liar dataset configer
# earlyStopAtTrainBatchIdx = 3400 # Set these variables to None to train and validate on all the dataset
# earlyStopAtValBatchIdx = 299 # Set these variables to None to train and validate on all the dataset

# twitter dataset configer
# earlyStopAtTrainBatchIdx = 850 # Set these variables to None to train and validate on all the dataset
# earlyStopAtValBatchIdx = 100 # Set these variables to None to train and validate on all the dataset



optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()
device="cuda"

for epoch in range(epochs):
    print("epochs: ", epoch)
    for batchIdx, sampledIdx in enumerate(tqdm(train_twitter_dataloader, position=0, leave=True)):
        # print("sampledIdx:: ",sampledIdx)
        sampledIdx = sampledIdx.cpu().data.numpy()
        model.train()
        optimizer.zero_grad()
        select_idxs= list(sampledIdx.flatten())
        # print("list(sampledIdx.flatten()) :: ",select_idxs)
        sampledRowText = list(data_twitter_clean["text"].iloc[select_idxs])
        # print("sampledRowText> \n ",sampledRowText)
        # print("len >  \n ",len(sampledRowText[2]))

        data_twitter_clean_types = data_twitter_clean["Type"].iloc[select_idxs]

        list_data_twitter_clean_types = list(data_twitter_clean_types)

        sampledRowLabels = torch.tensor(list_data_twitter_clean_types).to(device)
        # sampledRowLabels = torch.tensor(list(ipp)).to(device)
        encoded_input = tokenizer(sampledRowText, truncation=True, padding=True, return_tensors='pt').to(device) # Output shape: [bs, num_Labels]
        encoded_inputIds = encoded_input["input_ids"].to(device)
        encoded_attnMask = encoded_input["attention_mask"].to(device)
        # print("encoded_inputIds size >> ",len(encoded_inputIds[3]))
        outputs = model(input_ids=encoded_inputIds, attention_mask=encoded_attnMask)
        logits = outputs.logits
        loss = loss_fn(logits.squeeze(), sampledRowLabels)
        loss.backward()
        optimizer.step()

        if earlyStopAtTrainBatchIdx!=None:
            if batchIdx==earlyStopAtTrainBatchIdx: break
    # Validation
    predLs = []
    labelLs = []
    for batchIdx, sampledIdx in enumerate(eval_twitter_dataset):
        model.eval()

        sampledRowText = list(eval_data_twitter_clean["text"].iloc[list(sampledIdx.flatten())])
        sampledRowLabels = torch.tensor(list(eval_data_twitter_clean["Type"].iloc[list(sampledIdx.flatten())]))
        encoded_input = tokenizer(sampledRowText, truncation=True, padding=True, return_tensors='pt').to("cuda") # Output shape: [bs, num_Labels]
        encoded_inputIds = encoded_input["input_ids"].to("cuda")
        encoded_attnMask = encoded_input["attention_mask"].to("cuda")
        # print("encoded_inputIds size >> ",len(encoded_inputIds))
        outputs = model(input_ids=encoded_inputIds, attention_mask=encoded_attnMask)
        logits = outputs.logits
        predLs.append(torch.argmax(logits, dim=1).flatten().cpu().data.numpy())
        labelLs.append(sampledRowLabels.cpu().data.numpy())

        if earlyStopAtValBatchIdx!=None:
            if batchIdx==earlyStopAtValBatchIdx: break
    predLs = torch.tensor(predLs).flatten()
    labelLs = torch.tensor(labelLs).flatten()
    accuracy = Accuracy(task="multiclass", num_classes=2)
    valAcc = float(accuracy(predLs, labelLs))
    print("twitter type validation accuracy: ", valAcc)

epochs:  0


100%|██████████| 450/450 [01:11<00:00,  6.29it/s]


twitter type validation accuracy:  0.9354838728904724
epochs:  1


100%|██████████| 450/450 [01:11<00:00,  6.30it/s]


twitter type validation accuracy:  0.9354838728904724
epochs:  2


...

twitter type validation accuracy:  0.9354838728904724
epochs:  6


100%|██████████| 450/450 [01:11<00:00,  6.28it/s]


twitter type validation accuracy:  0.9354838728904724
epochs:  7


100%|██████████| 450/450 [01:10<00:00,  6.34it/s]


twitter type validation accuracy:  0.9354838728904724


Generating train split:   0%|          | 0/10269 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1283 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1284 [00:00<?, ? examples/s]

train
Dataset({
    features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
    num_rows: 10269
})
test
Dataset({
    features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
    num_rows: 1283
})
validation
Dataset({
    features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
    num_rows: 1284
})
 :  {'id': '2635.json', 'label': 0, 'statement': 'Says the Annies List political group supports third-trimester abortions on demand.', 'subject': 'abortion', 'speaker': 'dwayne-bohac', 'job_titl

<bound method DataFrame.info of        Unnamed: 0                                               text  label
0               0  Says the Annies List political group supports ...      0
1               1  When did the decline of coal start? It started...      1
2               2  Hillary Clinton agrees with John McCain "by vo...      1
3               3  Health care reform legislation is likely to ma...      0
4               4  The economic turnaround started at the end of ...      1
...           ...                                                ...    ...
10264       10264  There are a larger number of shark attacks in ...      1
10265       10265  Democrats have now become the party of the [At...      1
10266       10266  Says an alternative to Social Security that op...      1
10267       10267  On lifting the U.S. Cuban embargo and allowing...      0
10268       10268  The Department of Veterans Affairs has a manua...      0

[10269 rows x 3 columns]>
<bound method DataFrame.info 

,text,Type
2053,"""When I was in college, we used to take a popc...",1
5922,President Obamas proposal calls for serious cu...,1
7967,Dave Brat worked on Democrat Gov. Tim Kaines C...,1
6652,The Seattle Times called Patty Murray a prolif...,1
7549,Federal government reneging on Medicaid paymen...,0
...,...,...
4262,"Every taxpayer owes about $130,000 to pay off ...",1
8445,His biography says he was the top ROTC officer...,1
5608,Wisconsin is called the Badger State because o...,1
9690,Obamacare cuts $500 billion in future Medicare...,1


In [ ]:

# data_twitter_clean = clean(data_twitter)
# eval_data_twitter_clean = clean(eval_data_twitter)
# eval_data_twitter_clean

import pandas as pd
import numpy as np

train_data = pd.read_csv("./tweet_Constraint_Train.csv")#
train_data = train_data.sample(3000)
# test_data = pd.read_csv("./Constraint_Test.csv")
print("train_data :: \n ",train_data)
# print("test_data :: \n ",test_data)

# train_data['text']=train_data['tweet']
# train_data['Type']=train_data['label']

def change_csv(data,typestr):
  pp = data[data['label']==typestr]
  # print("train_data :: \n ",pp)
  if typestr=='real':
    pp['label']=1
  else:
    pp['label']=0
  # print("train_data :: \n ",pp)
  pp.rename(columns={'label':'Type','tweet':"text"},inplace=True)
  pp = pp[{'text','Type'}]
  # print("train_data :: \n ",pp.columns)
  # print("train_data :: \n ",pp)
  return pp

real = change_csv(train_data,'real')
fake = change_csv(train_data,'fake')
data_twitter = pd.concat([real,fake],axis=0)
print("dd is :::\n  ",data_twitter)
sum_idx = data_twitter.shape[0]
print('sum_idx :: ',sum_idx)
data_twitter = data_twitter.take(np.random.permutation(sum_idx))
print("twitter_data :: \n ",data_twitter)

earlyStopAtTrainBatchIdx = 850 # Set these variables to None to train and validate on all the dataset
earlyStopAtValBatchIdx = 100

eval_data_twitter = data_twitter.sample(300)
print("eval_twitter_data :: \n ",eval_data_twitter.index)
eval_index = eval_data_twitter.index
for e_idx in eval_index:
  # print(e_idx)
  data_twitter = data_twitter.drop(index=e_idx)

# data_train = data_index-eval_index
# print("twitter_data ::: \n",data_twitter)
# print(data_twitter.loc[0])


train_data :: 
          id                                              tweet label
855    856  Cuban researchers have developed a cure agains...  fake
3046  3047  About 75% of #COVID decedents had co-morbiditi...  real
4534  4535  Daily numbers from Texas included a backlog of...  real
4803  4804  Man Dying From Coronavirus Spends Final Day Ne...  fake
2118  2119  #CoronaVirusUpdates #IndiaFightsCorona India’s...  real
...    ...                                                ...   ...
5493  5494  COVID-19 Update: There are no new cases of COV...  real
2703  2704  The biggest Covid expert at Nanavati Hospital ...  fake
1068  1069  #NewJersey has moved to green on our COVID War...  real
5443  5444  From our perspective it is better to have all ...  real
3014  3015  📢#CoronaVirusUpdates: ✅India's #COVID19 recove...  real

[3000 rows x 3 columns]
dd is :::
         Type                                               text
3046     1  About 75% of #COVID decedents had co-morbiditi...
4534 

In [ ]:
#释放GPU内存
import torch
import gc

# model.cpu()
# del model, tokenizer
gc.collect()
torch.cuda.empty_cache()


# model = None
# tokenizer = None

def free():
  import torch
  import gc
  # model.cpu()
  del model, tokenizer
  gc.collect()
  torch.cuda.empty_cache()

In [ ]:
model.save_pretrained("train_model")

In [ ]:
lora_model.save_pretrained("./train_lora")

In [ ]:
import pandas as pd
# fake news long text more 500 words
data = pd.read_csv('./long_True.csv')
fake = pd.read_csv('./long_Fake.csv')
# print(data)

data = data.sample(1000)
fake = fake.sample(1000)
data['text']=data['title']+". "+ data['text']
fake['text']=fake['title']+". "+ fake['text']
data["Type"] = 1
fake["Type"] = 0
data = pd.concat([data,fake],axis=0)
data = data.sample(2000,random_state=999)
# data=data['text']
# print(data)
def getsize(v):
  texts = v.split()
  return len(texts)

def getlen(v):
  return len(v)

print(data)


                                                   title  \
21042  MIGRANT GIRLS AS YOUNG AS 11 Arrive In Sweden ...   
7107    The Panama Papers: How The 1%’s Greed Has Lit...   
112    White House optimistic Congress will avoid gov...   
5765   Medical group urges protection for doctors, pa...   
3248    Trump Gets HUMILIATED After His Latest Stupid...   
...                                                  ...   
12991  SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...   
3644   Trump asked two top intelligence officials to ...   
18916  Eating leaves to survive in Myanmar's 'ethnic ...   
6150   Trump son-in-law Kushner can serve as White Ho...   
15913  OBAMA HID BIN LADEN PAPERS To Bury This Damnin...   

                                                    text          subject  \
21042  MIGRANT GIRLS AS YOUNG AS 11 Arrive In Sweden ...        left-news   
7107    The Panama Papers: How The 1%’s Greed Has Lit...             News   
112    White House optimistic Congress will avoi

In [55]:
from transformers import pipeline


def summarize_text(text,summarizer, max_length=200):
    texts = text.split()
    # print("text:: ",text)
    # print("len:: ",len(texts))
    if len(texts)<=max_length:
      return text
    if summarizer==None:
      summarizer = pipeline("summarization")
    minstr =" "
    try :
      summary = summarizer(text, max_length=max_length, min_length=60, length_penalty=2.0, max_time=120)
      minstr = summary[0]['summary_text']
    except:
      # print("len:: in except ..")
      minstr = " ".join(texts[0:max_length-1])
    else:
      pass
    return minstr



# summarizer the text
summarizer = pipeline("summarization")


def summerize_text_data(data,summarizer):
  data["text"] = data["text"].apply(lambda x:summarize_text(x,summarizer,max_length=200))
  data=data[["text",'Type']]
  return data

data = summerize_text_data(data,summarizer)


print(data)



No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Token indices sequence length is longer than the specified maximum sequence length for this model (1244 > 1024). Running this sequence through the model will result in indexing errors


                                                    text  Type
21042   Of the 61 married minors who sought asylum in...     0
7107    The Panama Papers: How The 1%’s Greed Has Lit...     0
112    White House optimistic Congress will avoid gov...     1
5765    American Medical Association urges protection...     1
3248   Trump Gets HUMILIATED After His Latest Stupid-...     0
...                                                  ...   ...
12991   Hillary Clinton ally David Brock is offering ...     0
3644   Trump asked two top intelligence officials to ...     1
18916  Eating leaves to survive in Myanmar's 'ethnic ...     1
6150    U.S. Department of Justice: President can hir...     1
15913   The Obama administration hand-picked just a f...     0

[2000 rows x 2 columns]


In [56]:
data.to_csv("./long_fake_news_after_summerize.csv")
data

In [32]:
import pandas as pd
import random
# print(data.shape)
# print(data.shape[0])
# print(type(data.shape[0]))
data = pd.read_csv("./long_fake_news_after_summerize.csv")
data = data.sample(1500)
# print(data.index)
def split_train_test(data):
  import pandas as pd
  import random
  test_number = data.shape[0]*0.1
  test = []
  for _ in range(int(test_number)):
    random_index = random.choice(data.index)
    random_sample = data.loc[random_index]
    test.append(random_sample)
    data = data.drop(index=random_index)
  test_pd = pd.DataFrame(test)
  return data,test_pd

data_twitter,eval_data_twitter = split_train_test(data)

print(data_twitter)
print(eval_data_twitter)

      Unnamed: 0                                               text  Type
1385        2863   Republicans voice growing doubts on U.S. heal...     1
852         3896   Reports are surfacing that, in the 11th hour,...     0
144        11740   Federal Judge James Robart, a George W. Bush ...     0
637        17969  EU says 'ball entirely in UK court' for move t...     1
822        15049   U.S. President Donald Trump offers to mediate...     1
...          ...                                                ...   ...
1285        8650  Michigan Gov. Poison Gets Humiliated By Mark R...     0
103        15650  Trump in Japan stresses unity on North Korea; ...     1
1259       20631   Central African Republic risks return to majo...     1
711         6339   An email obtained by WTVF in Nashville, Tenne...     0
1027        1180   Trump administration again declines to name C...     1

[1350 rows x 3 columns]
      Unnamed: 0                                               text  Type
1802        1

In [ ]:
print(data_twitter['Type'])
print(eval_data_twitter['Type'])

17534    1
1529     1
9405     0
11699    1
20973    1
        ..
7502     1
17448    0
6658     0
5805     0
18659    0
Name: Type, Length: 360, dtype: object
18965    0
16267    1
831      1
18079    1
8012     1
17513    0
11968    1
1975     0
22515    0
5791     1
19826    1
16183    1
16028    0
12999    1
186      1
22354    0
20942    1
17990    0
4215     0
21798    0
2242     1
3266     1
1611     0
18646    1
4984     1
11528    0
14629    0
19145    0
4910     1
14143    1
12822    0
3883     1
13565    1
258      0
18630    0
13033    1
14997    1
8171     0
15301    1
19344    0
Name: Type, dtype: object
